# Q&A against Tabular Data from a CSV file  (experimental)

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  **The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.**

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

**Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question.** A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [1]:
import os
import pandas as pd

from langchain.chat_models import AzureChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.agents.agent_toolkits import create_csv_agent

from common.prompts import CSV_PROMPT_PREFIX, CSV_PROMPT_SUFFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

## Download the dataset and load it into Pandas Dataframe

In [3]:
os.makedirs("data",exist_ok=True)

In [4]:
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

--2024-09-22 20:52:13--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 172.67.183.132, 104.21.64.114, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|172.67.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv.1’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.04s   

2024-09-22 20:52:13 (62.5 MB/s) - ‘./data/all-states-history.csv.1’ saved [2738601]



In [6]:
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (20780, 41)


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [6]:
df.columns

Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'

## CSV/Pandas Agent

LLMs are great for building question-answering systems over various types of data sources. In this section we’ll go over how to build Q&A systems over data stored in a CSV file(s). Like we did in the last notebook, the key to working with tabular CSV files is to give an LLM access to tools/experts for querying and interacting with the data

In [2]:
# Let's delve into a challenging question that demands a multi-step solution. 
# The path to solving it might not be immediately clear.
# When examining the dataframe above, even a human might struggle to determine which columns are pertinent.

QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
Use the hospitalizedIncrease column
"""

In [4]:
from azure.identity import ManagedIdentityCredential
from langchain_openai import AzureChatOpenAI
#from langchain.chat_models import AzureChatOpenAI
import openai

# Step 1: Specify the client ID of the User Managed Identity
user_managed_identity_client_id = "d30cba06-04c1-4065-a91d-8b7ce3b07b78"  # Replace with your User Managed Identity client ID

# Step 2: Fetch the access token using ManagedIdentityCredential and the client ID of the user-managed identity
credential = ManagedIdentityCredential(client_id=user_managed_identity_client_id)
token = credential.get_token("https://cognitiveservices.azure.com/.default")

# Step 3: Set the access token in the OpenAI API
openai.api_key = token.token
openai.api_type = "azure"
openai.api_base = "https://azuremlopenai.openai.azure.com/"  # Replace with your OpenAI resource's base URL
openai.api_version = "2023-06-01-preview"  # Use the correct API version

# Step 4: Use AzureChatOpenAI with Langchain
llm = AzureChatOpenAI(
    openai_api_key=token.token,  # Pass the managed identity token as the API key
    deployment_name="gpt-4o",  # Replace with your deployment name
    azure_endpoint=openai.api_base,  # Pass the API base URL
    openai_api_version=openai.api_version,  # Pass the API version
    temperature=0, max_tokens=500
)

# Now you can use the chat model like any other Langchain model
response = llm.invoke(QUESTION)
print(response)


content='To provide the specific number of patients hospitalized during July 2020 in Texas and nationwide using the "hospitalizedIncrease" column, I would need access to the COVID-19 hospitalization data from that period. This data is typically available from health departments or organizations that track COVID-19 statistics, such as the COVID Tracking Project or the Centers for Disease Control and Prevention (CDC).\n\nHowever, I can guide you on how to find this information:\n\n1. **COVID Tracking Project**: This project was a reliable source for COVID-19 data in the United States until it stopped collecting data in March 2021. You can access their historical data on GitHub or their website archives.\n\n2. **CDC COVID Data Tracker**: The CDC provides comprehensive data on COVID-19, including hospitalizations. You can visit their website and use their data tracker to find historical data.\n\n3. **State Health Departments**: For Texas-specific data, you can visit the Texas Department of

Now we **need our agent and our expert/tool.**  
LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem. For more informatio about tje **CSV Agent** click [HERE](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html)

In [22]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               verbose=True,
                                               agent_type="openai-tools")

In [15]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['date'].str.startswith('2020-07') & (df['state'] == 'TX')]['hospitalizedIncrease'].sum()"}`


0
Invoking: `python_repl_ast` with `{'query': "df[df['date'].str.startswith('2020-07')]['hospitalizedIncrease'].sum()"}`


63105In July 2020, there were 0 patients hospitalized in Texas according to the `hospitalizedIncrease` column. Nationwide, the total of all states was 63,105 hospitalized patients.

> Finished chain.


In July 2020, there were 0 patients hospitalized in Texas according to the `hospitalizedIncrease` column. Nationwide, the total of all states was 63,105 hospitalized patients.

CPU times: user 175 ms, sys: 15.9 ms, total: 190 ms
Wall time: 2.01 s


## Enjoy the response and the power of GPT-4 + REACT/MKRL approach

In [10]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        continue
        
print(response)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: First, I need to set the pandas display options to show all columns. Then, I will filter the dataframe for the state of Texas and the month of July 2020. I will sum the `hospitalizedIncrease` column for these filtered rows to get the number of hospitalized patients in Texas. Similarly, I will filter the dataframe for the month of July 2020 and sum the `hospitalizedIncrease` column for all states to get the nationwide total.

Action: Set pandas display options to show all columns and get the column names.
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
df.columns
```Set pandas display options to show all columns and get the column names. is not a valid tool, try one of [python_repl_ast].I need to set the pandas display options to show all columns and then get the column names to understand the structure of the dataframe. After that, I will filter the dataframe for the state of Texas and the month of

That was good pandas code, and pretty fast. Let's add some prompt engineering to improve answer.    

### Improving the Prompt

We can give it now an improved sager prompt. In `prompts.py` you can find `CSV_PROMPT_PREFIX`

In [ ]:
CSV_PROMPT_PREFIX

In [ ]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               prefix=CSV_PROMPT_PREFIX,
                                               verbose=True,
                                               agent_type="openai-tools")

In [ ]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)

## Evaluation
Let's see if the answer is correct

In [11]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [12]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, LLMs can do SQL very well. 


**Note**: We don't recommend using a pandas agent to answer questions from tabular data. It is not fast and it makes too many parsing mistakes. We recommend using SQL (see next notebook).

# NEXT
We can see that GPT-3.5, and even better GPT-4, is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.